In [103]:
from common_funcs import *
import string, re, emoji
from langdetect import detect
from autocorrect import Speller
from nltk.tokenize import sent_tokenize

In [22]:
# Note, import text_df pickle if restarting kernel. Langdetect takes a long time.

In [3]:
text_df = pd.read_pickle('data/text_df')
text_df.head()

,country,comments_or_subs,democracy_score,GDP,abstained,text
0,india,comments,6.91,2946061.0,True,"&gt; Indian students walk out of Kharkiv, as b..."
1,india,comments,6.91,2946061.0,True,The Kremlin readout of the Modi-Putin telephon...
2,india,comments,6.91,2946061.0,True,"Russia is part of the UNGA, everyone in the bo..."
3,india,comments,6.91,2946061.0,True,Hopefully one thing this tradgedy has taught u...
4,india,comments,6.91,2946061.0,True,Look at a map of India and its surrounding cou...


# Text Cleaning
## Remove URLs
These do not add anything to the analysis.

In [4]:
# https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
def remove_urls(text):
    text = re.sub(r'https?://\S+', '', text)
    return text

In [5]:
text_df.text = text_df.text.apply(remove_urls)

## Detect language
This will be important for things like spell check and emoji processing to know what language a post is written in.

In [9]:
def detect_language(text):
    try:
        text = text[:100] # truncate text to speed up algorithm
        lang = detect(text)
        return lang
    except:
        return 'unknown'

In [10]:
# Note, this will take some time to run
text_df['language'] = text_df.text.apply(detect_language)

In [13]:
text_df.to_pickle('data/text_df')

## Remove punctuation
These interfere with the vectorizer.

In [16]:
def remove_punctuation(text):
    text = text.replace('\n', ' ')
    return re.sub('[%s]' % re.escape(string.punctuation), ' ', text)

In [18]:
text_df.text = text_df.text.apply(remove_punctuation)

## Replace emojis with text
I will replace emojis with English or whatever the language of the post is to make text analysis easier. I will follow the instructions from [here](https://stackoverflow.com/questions/57580288/how-to-replace-emoji-to-word-in-a-text).

In [31]:
def clean_emojis(text, language):
    if len(language) > 2:
        language = 'en'
        
    # remove emojis and set delimiters to "" because otherwise it will put ':translation:' instead of 'translation'
    text = emoji.demojize(text, language = language, delimiters=("", "")) 
    text = text.replace('_', ' ')
    return text

In [42]:
text_df.text = text_df.apply(lambda x: clean_emojis(x.text, x.language), axis=1).copy()

In [13]:
text_df.to_pickle('data/text_df_cleaned')

## Convert all text to lower case
Will make vectorizing easier

In [45]:
text_df.text = text_df.text.str.lower()

## Spell check

In [65]:
def spell_check(text, language):
    if language not in 'en, pl, ru, uk, tr, es, pt, cs, el, it, fr, vi'.split(', '): # these are supported langs
        return text
    spell = Speller(language)
    return spell(text)

In [73]:
%%time
counter=0
# 257,562 rows takes 10 hours to run
for row in range(len(text_df)):
    language = text_df.loc[row, 'language']
    if language in 'en, pl, ru, uk, tr, es, pt, cs, el, it, fr, vi'.split(', '): # these are supported langs
        text_df.loc[row, 'text'] = spell_check(text_df.loc[row, 'text'], language)
    counter += 1
    if counter%1000==0: print(counter)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [74]:
text_df.to_pickle('data/text_df_cleaned')

## Remove numbers
Numbers do not contribute to the text analysis.

In [79]:
def remove_numbers(text):
    return re.sub(r'[0-9]+', '', text)

In [82]:
text_df.text = text_df.text.apply(remove_numbers)

In [83]:
text_df.to_pickle('data/text_df_cleaned')

Now moving on further pre-processing by removing stop words and lemmatizing in the next notebook.

## Spell check only
All that cleaning could impact the sentiment scores unnecessarily so we will redo the spell check, then translate to English, then run sentiment analysis on the results. Before spell check, we will remove hyperlinks, numbers, and detect language.

In [97]:
text_df = pd.read_pickle('data/text_df')

In [99]:
text_df.text = text_df.text.apply(remove_urls)

In [100]:
text_df.text = text_df.text.apply(remove_numbers)

In [101]:
%%time
counter=0
# 257,562 rows takes 14 hours to run with less cleaning
for row in range(len(text_df)):
    language = text_df.loc[row, 'language']
    if language in 'en, pl, ru, uk, tr, es, pt, cs, el, it, fr, vi'.split(', '): # these are supported languages
        text_df.loc[row, 'text'] = spell_check(text_df.loc[row, 'text'], language)
    counter += 1
    if counter%10000==0: 
        print(counter)
        text_df.to_pickle('data/text_df_spelling')
        
text_df.to_pickle('data/text_df_spelling')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
CPU times: user 13h 15min 57s, sys: 4min 51s, total: 13h 20min 48s
Wall time: 13h 57min 59s


Before running translation, we will sentence-tokenize and translate each sentence. That will be followed by sentiment analysis in the next notebook.

In [147]:
text_df_spelling_sent_tokenized = pd.DataFrame(columns = text_df.columns)

In [148]:
# These are the columns that will stay the same for each sentence in a post
cols = ['country', 'comments_or_subs', 'democracy_score', 'GDP', 'abstained', 'language']

In [149]:
%%time
counter = 0

# This will 8 hours to run

for row in range(len(text_df)):
    sentences = sent_tokenize(text_df.loc[row, 'text'])
    for sentence in sentences:
        sub_df = pd.DataFrame(text_df.loc[row, cols]).transpose()#, columns = text_df.columns)
        sub_df.loc[row, 'text'] = sentence
        text_df_spelling_sent_tokenized = pd.concat([text_df_spelling_sent_tokenized, sub_df])

    counter += 1
    if counter%1000==0:
        print(counter)
        text_df_spelling_sent_tokenized.to_pickle('data/text_df_spelling_sent_tokenized')
#         break
text_df_spelling_sent_tokenized.to_pickle('data/text_df_spelling_sent_tokenized')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [151]:
text_df_spelling_sent_tokenized.shape

(940857, 7)

Now that sentence tokenizing is done, we will get the VADER sentiment analysis for each sentence in the next notebook. After that, we will proceed to text analysis.